This notebook extracts the resting-state temporal dynamics of brain networks.


We want to construct a reasonably realistic model of interacting regions or nodes. We first use the atlas to mask BOLD activities of the nodes, remove confounds from the data, and extract the time-series for each node. This process ends up with a matrix of time-series of size n_subjects * n_nodes * n_timesteps, e.g., 32*39*125 in case of using \cite{@dosenbach2007} atlas, or 32*1024*125 when using DiFuMo/1024/2 atlas.

DiFuMo article: https://doi.org/10.1016/j.neuroimage.2020.117126


**WARNING**: *This notebook requires full BIDS dataset including fmriprep derivatives and preprocessed scans.*

In [1]:
import numpy as np
import xarray as xr
import pandas as pd
from pathlib import Path

from bids import BIDSLayout

import nilearn
from nilearn import datasets, plotting, regions, maskers, image, connectome


In [2]:
# DIFUMO Atlas

atlas = datasets.fetch_atlas_difumo(dimension=1024, resolution_mm=2, legacy_format=False)
atlas_coordinates = nilearn.plotting.find_probabilistic_atlas_cut_coords(maps_img=atlas.maps)

# DEBUG atlas.keys()
# DEBUG nilearn.plotting.plot_prob_atlas(atlas.maps, title='DIFUMO atlas')

In [3]:
# Yeo2011 Atlas

# Steps: 
# 1. fetch atlas, 2. relabel atlas for continuity, 3. resample images, 4. mask and extract time series

# atlas = datasets.fetch_atlas_yeo_2011()['thick_7'],
# merge voxels belonging to the same network based on spatial continuity
# atlas_relabeled = nilearn.regions.connected_label_regions(atlas)
# nilearn.plotting.plot_roi(
#   atlas_relabeled,
# 	cut_coords=range(-30,70,10),
# 	display_mode='z',
# 	title='Yeo Atlas (voxels are re-labeled for continuity)')
# atlas_coordinates = plotting.find_parcellation_cut_coords(labels_img=atlas_relabeled)
# atlas_resampled = nilearn.image.resample_to_img(atlas_relabeled, func)


In [4]:
# load rs-fMRI session from the BIDS dataset

layout = BIDSLayout('data/julia2018_x5alias',
                    derivatives=True,
                    database_path='tmp/pybids_cache/julia2018')

# all_subjects = layout.get_subject()
TR = layout.get_tr(task='rest')

images = layout.get(task='rest',
                    desc='preproc',
                    suffix='bold',
                    scope='fmriprep',
                    extension='nii.gz',
                    return_type='filename')

mask_images = layout.get(task='rest',
                         desc='brain',
                         suffix='mask',
                         scope='fmriprep',
                         extension='nii.gz',
                         return_type='filename')

confounds_files = layout.get(task='rest',
                             desc='confounds',
                             suffix='timeseries',
                             scope='fmriprep',
                             extension='tsv',
                             return_type='filename')

confounds_cols = ['trans_x', 'trans_y', 'trans_z',
                  'csf', #'whitematter',
                  'rot_x', 'rot_y', 'rot_z',
                  'global_signal',
                  'a_comp_cor_00', 'a_comp_cor_01',
                  ]

In [5]:

# extract resting timeseries
import re
from tqdm import tqdm

timeseries = {}
subjects = []

for img, mask_img, confounds in tqdm(zip(images, mask_images, confounds_files)):

    subject = re.search('func/sub-(.*)_ses', img)[1]
    subjects.append(subject)
    
    output_path = Path('tmp/difumo1024_cache/') / f'{subject}_timeseries.npz'

    if output_path.exists():
        # print(f'Loading {subject} from cache...')
        ts = np.load(output_path)['arr_0']
        timeseries[subject] = ts
        continue

    # yeo2011
    # resampled_atlas = nilearn.image.resample_to_img(atlas_relabeled, img)

    masker = nilearn.maskers.NiftiMapsMasker(
        atlas.maps,
        mask_img=mask_img,
        detrend=True,
        standardize=True,
        t_r = TR,
        verbose=0)

        # for some reasons, using cache causes signal extraction to fail
        # memory='tmp/nilearn_cache', memory_level=1,

    confounds = pd.read_csv(confounds, sep='\t')[confounds_cols].values
    ts = masker.fit_transform(img, confounds=confounds)
    
    np.savez(output_path, ts)
    timeseries[subject] = ts

# reshape dim-0 is subject, dim-1 is region, dim-2 is time point
timeseries = np.array(list(timeseries.values())).transpose(0,2,1)


32it [00:00, 219.19it/s]


In [6]:
# DEBUG: plot connectome

# conn = nilearn.connectome.ConnectivityMeasure(kind='correlation')
# conn.fit_transform(timeseries.values())

# np.fill_diagonal(conn.mean_, 0)

# drop missing yeo2011-based labels
# node_coords = pd.DataFrame(atlas_coordinates).drop(index=[32, 3, 4, 5, 39, 9, 11, 48]).values
# node_coords = atlas_coordinates  # difumo

# # skip the background label (index-0)
# # plotting.plot_matrix(conn.mean_, labels=list(range(1,42)), vmax=0.8, vmin=-0.8, reorder=True)

# # plot connectome with 80% edge strength in the connectivity
# plotting.plot_connectome(conn.mean_, node_coordinates,
#                          edge_threshold="95%",
#                          title='Average connectivity in DIFUMO 1024/2 atlas')

In [7]:
atlas_regions = pd.concat([atlas.labels, pd.DataFrame(atlas_coordinates)], axis=1)
atlas_regions.rename(columns={0: 'mni152_x',
                              1: 'mni152_y',
                              2: 'mni152_z'}, inplace=True)

atlas_regions.drop(columns=['component'], inplace=True)
atlas_regions.index.name = 'region'

ds = xr.Dataset({
  'timeseries': (['subject', 'region', 'timestep'], timeseries)
})

# metadata
ds.coords['timestep']  = range(0,125)
ds.attrs['description'] = 'Resting-state time series from fmriprep scans based on DIFUMO atlas'
ds.coords['subject'].attrs['description'] = ('subject identifier; AVGPxx for action video'
                                             'gamers and NVGPxx for non-video gamers')

# subject data
bids_participants = pd.read_csv('data/julia2018/participants.tsv', sep='\t')
bids_participants.rename(columns={'participant_id': 'subject'}, inplace=True)
bids_participants['subject'] = bids_participants['subject'].apply(
  lambda x: x.split('-')[1])
bids_participants.set_index('subject', inplace=True)
bids_participants = bids_participants.query('index in @subjects')

# merge all datasets and store
ds = xr.merge([ds, bids_participants.to_xarray(), atlas_regions.to_xarray()])
ds.to_netcdf('data/julia2018_resting/timeseries_difumo_1024_2.nc', engine='netcdf4')
ds

<xarray.Dataset>
Dimensions:                      (subject: 32, region: 1024, timestep: 125)
Coordinates:
  * subject                      (subject) object 'AVGP01' ... 'NVGP19NEW'
  * region                       (region) int64 0 1 2 3 ... 1020 1021 1022 1023
  * timestep                     (timestep) int64 0 1 2 3 4 ... 121 122 123 124
Data variables: (12/18)
    timeseries                   (subject, region, timestep) float64 -0.8065 ...
    age                          (subject) float64 27.0 19.0 23.0 ... 19.0 22.0
    sex                          (subject) object 'M' 'M' 'M' ... 'M' 'M' 'M'
    group                        (subject) object 'AVGP' 'AVGP' ... 'NVGP'
    preprocessed_beh             (subject) bool True True True ... True True
    preprocessed_rsfmri          (subject) bool True True True ... True True
    ...                           ...
    gm                           (region) float64 0.4481 0.01733 ... 0.2119
    wm                           (region) float64 0.5365 0.9827 ... 0.5305
    csf                          (region) float64 0.0154 2.366e-05 ... 0.2576
    mni152_x                     (region) float64 24.04 27.11 ... -13.35 12.54
    mni152_y                     (region) float64 -90.22 -30.03 ... 6.905 28.26
    mni152_z                     (region) float64 -7.482 31.32 ... 18.44 5.24
Attributes:
    description:  Resting-state time series from fmriprep scans based on DIFU...